<a href="https://colab.research.google.com/github/alickchoi/STAT5106-Lab/blob/main/Week_5_Web_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Connecting to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

import os

# Your original folder
gdrive_folder = 'CUHK_STAT5106_2025/Week 5 - Oct 9'
os.chdir(f'/content/gdrive/My Drive/{gdrive_folder}')

# check whether the folder is as your expected
print(os.listdir())

Mounted at /content/gdrive
['Week 5 - Google API Example.ipynb', 'Week 5 - Web Scraping.ipynb', 'Week 5.gslides']


# Web Scraping

In [ ]:
import urllib.request
import os
from bs4 import BeautifulSoup
import time

In [ ]:
print(os.getcwd())
# os.chdir('...')

/content/gdrive/MyDrive/CUHK_STAT5106_2025/Week 5 - Oct 9


## Example, back to Dept History

In [ ]:
# url = 'https://www.sta.cuhk.edu.hk/about-us/department-history/'
url = 'https://kyalan.github.io/kytest/department-history.html'
req = urllib.request.Request(url)
resp = urllib.request.urlopen(req)

In [ ]:
content = resp.read()

# If needed to print or process the content, you can do so here
print(content)

In [ ]:
# print(dir(req))
resp.__dict__

In [ ]:
# Or directly
resp = urllib.request.urlopen(url)

html = ''
if resp.status==200:   html = resp.read()
print(len(html))

In [ ]:
html[:100]

In [ ]:
type(html)

In [ ]:
soup = BeautifulSoup(html, 'html.parser')
tag_table = soup.table
print(tag_table)

In [ ]:
soup.find_all('tr')

In [ ]:
# Prettify the BeautifulSoup object: pretty_soup
print(tag_table.prettify())

In [ ]:
tag_rows = tag_table.findAll('tr', {'class':'odd'})
print(tag_rows)

We need to take out all words standing for "Professor", including "Prof.".

In [ ]:
import re

for tag in tag_rows:
  tag_values = tag.find_all('td')
  year = tag_values[0].text
  message = re.findall(r'Prof(?:essor)?\.?(?:\s[A-Z][a-z-]*)+', tag_values[1].text)
  print(year, message)

## Example on Fitness Rooms in LCSD site

![yuenchaukok_gym.png](https://drive.google.com/uc?export=view&id=1451sSsaHw1ugc2b1CRTxqHMjfYYCYxMu)

Gym rooms are provided by the Leisure and Cultural Services Department (LCSD) in the urban area and the New Territories. Let's access into the information site: https://www.lcsd.gov.hk/clpss/en/webApp/FitnessRooms.do

In [ ]:
import urllib.request, urllib.parse
from urllib.parse import urljoin
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
url = 'https://www.lcsd.gov.hk/clpss/en/webApp/FitnessRooms.do'
resp = urllib.request.Request(url=url, headers={'User-Agent': 'Mozilla/5.0'})

html = ''
req = urllib.request.urlopen(resp)
if req.status==200:   html = req.read().decode()
print(f'website retrieved: {len(html)}')

Note !

```
headers={'User-Agent': 'Mozilla/5.0'}
```
is a must here.


In [ ]:
soup = BeautifulSoup(html, 'html.parser')

We aim to obtain all rows of tables - \<tr\> tags and \<td\> for each tag

In [ ]:
tags_tr = soup.find_all('tr')
len(tags_tr)

In [ ]:
tags_td = tags_tr[1].find_all('td')

In [ ]:
tags_td

In [ ]:
rows = []
for tag_tr in tags_tr:
  tags_td = tag_tr.find_all('td')
  row = []
  for tag_td in tags_td:
    value = tag_td.text.strip()
    row.append(value)
  rows.append(row)

In [ ]:
rows

The link in html may be in form of relative link, says "/clpss/en/webApp/FitnessRoomDetails.do?id=1374" <br>
You can use urljoin with serviceurl = "https://www.lcsd.gov.hk" for making a full link.

In [ ]:
serviceurl = 'https://www.lcsd.gov.hk'

dict_tags = []
for tag in tags_tr:
    tags_td = tag.find_all('td')
    if len(tags_td) != 4: continue
    dict_tag = {}
    dict_tag['venu'] = tags_td[0].text
    dict_tag['addr'] = tags_td[1].text
    dict_tag['tel'] = tags_td[2].text
    dict_tag['link'] = urljoin(serviceurl, tags_td[3].a.get('href'))
    dict_tags.append(dict_tag)

In [ ]:
dict_tags

In [ ]:
df_url = pd.DataFrame(dict_tags)

In [ ]:
df_url

# Why google search url cannot be worked?

In [ ]:
# url = 'https://www.google.com/search?q=japanese+onsen&tbm=isch'
url = 'https://www.google.com/search?q=japanese+onsen'
req = urllib.request.urlopen(url)

In [ ]:
req.read()

Also, some of sites do not work...

In [ ]:
url = 'https://www.skyscanner.com.hk/transport/flights/hkg/lond/?null&oym=2512&selectedoday=21&iym=2512&selectediday=31'
req = urllib.request.urlopen(url)
html = req.read()

In [ ]:
print(BeautifulSoup(html, 'html.parser').prettify())

tag list: https://www.w3schools.com/tags/ref_byfunc.asp